In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import nept

from loading_data import get_data
from utils_maze import get_bin_centers, get_subset_zones, get_trials, get_xy_idx

In [ ]:
import info.r063d7 as r063d7
import info.r063d8 as r063d8
# infos = [r063d7, r063d8]
from run import analysis_infos
infos = analysis_infos

In [ ]:
min_samples = 50
len_min_trials = 30

maze_segments = ["u", "shortcut", "novel"]

all_n_trials = {key: [] for key in maze_segments}
all_trial_duration = {key: [] for key in maze_segments}
all_trial_order = {key: [] for key in maze_segments}

for info in infos:
    events, position, _, _, _ = get_data(info)

    phase = info.task_times["phase3"]
    trials = get_trials(events, phase)

    u_zone, shortcut_zone, novel_zone = get_subset_zones(info, position)

    n_trials = {key: 0 for key in maze_segments}
    trial_duration = {key: [] for key in maze_segments}
    trial_order = {key: np.zeros(len_min_trials) for key in maze_segments}

    for i, (trial_start, trial_stop) in enumerate(zip(trials.starts, trials.stops)):
        x_idxs, y_idxs = get_xy_idx(info, position.time_slice(trial_start, trial_stop))

        if np.sum(novel_zone[y_idxs, x_idxs]) > min_samples:
            n_trials["novel"] += 1
            trial_duration["novel"].append(trial_stop-trial_start)
            if i < len_min_trials:
                trial_order["novel"][i] = 1.0
        elif np.sum(shortcut_zone[y_idxs, x_idxs]) > min_samples:
            n_trials["shortcut"] += 1
            trial_duration["shortcut"].append(trial_stop-trial_start)
            if i < len_min_trials:
                trial_order["shortcut"][i] = 1.0
        elif np.sum(u_zone[y_idxs, x_idxs]) > min_samples:
            n_trials["u"] += 1
            trial_duration["u"].append(trial_stop-trial_start)
            if i < len_min_trials:
                trial_order["u"][i] = 1.0
    for maze_segment in maze_segments:
        all_n_trials[maze_segment].append(n_trials[maze_segment])
        all_trial_duration[maze_segment].extend(trial_duration[maze_segment])
        all_trial_order[maze_segment].append(trial_order[maze_segment])

In [ ]:
n_bars = np.arange(3)
plt.bar(n_bars, [n_trials[maze_segment] for maze_segment in maze_segments])
plt.xticks(n_bars, maze_segments)
plt.title("N trials for "+ info.session_id)
plt.show()

In [ ]:
n_bars = np.arange(3)
plt.bar(n_bars, [np.mean(all_n_trials[maze_segment]) for maze_segment in maze_segments])
plt.xticks(n_bars, maze_segments)
plt.title("N trials for "+ info.session_id)
plt.show()

In [ ]:
plt.boxplot([trial_duration[maze_segment] for maze_segment in maze_segments])
plt.show()

In [ ]:
fig, ax = plt.subplots()
bp = plt.boxplot([trial_duration[maze_segment] for maze_segment in maze_segments], patch_artist=True)

## change outline color, fill color and linewidth of the boxes
for box in bp['boxes']:
    # change outline color
    box.set(color='#7570b3', linewidth=2)
    # change fill color
    box.set(facecolor = '#1b9e77')

## change color and linewidth of the whiskers
for whisker in bp['whiskers']:
    whisker.set(color='#7570b3', linewidth=2)

## change color and linewidth of the caps
for cap in bp['caps']:
    cap.set(color='#7570b3', linewidth=2)

## change color and linewidth of the medians
for median in bp['medians']:
    median.set(color='#b2df8a', linewidth=2)

## change the style of fliers and their fill
for flier in bp['fliers']:
    flier.set(marker='o', color='#e7298a', alpha=0.5)
    
ax.set_xticklabels(maze_segments)
plt.title("Trial duration for "+ info.session_id)
plt.show()

In [ ]:
fig, ax = plt.subplots()
bp = plt.boxplot([all_trial_duration[maze_segment] for maze_segment in maze_segments], patch_artist=True)

## change outline color, fill color and linewidth of the boxes
for box in bp['boxes']:
    # change outline color
    box.set(color='#7570b3', linewidth=2)
    # change fill color
    box.set(facecolor = '#1b9e77')

## change color and linewidth of the whiskers
for whisker in bp['whiskers']:
    whisker.set(color='#7570b3', linewidth=2)

## change color and linewidth of the caps
for cap in bp['caps']:
    cap.set(color='#7570b3', linewidth=2)

## change color and linewidth of the medians
for median in bp['medians']:
    median.set(color='#b2df8a', linewidth=2)

## change the style of fliers and their fill
for flier in bp['fliers']:
    flier.set(marker='o', color='#e7298a', alpha=0.5)
    
ax.set_xticklabels(maze_segments)
plt.title("Trial duration for all sessions")
plt.show()

In [ ]:
plt.plot(np.mean(all_trial_order["u"], axis=0), "b")
plt.plot(np.mean(all_trial_order["shortcut"], axis=0), "g")
plt.plot(np.mean(all_trial_order["novel"], axis=0), "r")
plt.legend(maze_segments)
plt.title("Choice over time")
plt.show()

In [ ]:
for trial_start, trial_stop in zip(trials.starts[:10], trials.stops[:10]):

    tt = position.time_slice(trial_start, trial_stop)

    plt.plot(position.x, position.y, "y.")
    plt.plot(tt.x, tt.y, "k.", ms=4)
    plt.show()